### CoreBx_island_refac - Interpolate the North Core Banks DEMs onto rotated 1-m grid and save each as a .nc file.

Versioning jumped from v6 to refac

New invV5
* Files are switched to the "merged DEMs" that Jin-Si made, so the rapid iteration can occur.
* Box is re-adjusted to accomodate the whole island. The resulting array is huge, but manageble.

New in v2
* Now 4D maps, two made made during visit to Santa Cruz and two ftp'd from Andy
* Apr. 9 - changed to _v3 for Sep map
* Now does the interpolation without the loop
* Apr. 21 - moved origin to SE to accomodate curvature in NE end of island. Add 400 m to size of array.
* Watch file names, esp. underline (or not) after "1m_DEM"

New in v6
* Added maps through Sep 28 2020

New in refac  
* Added newest DEMs, but they have not been clipped.
* Code is now maintained in ../src/CoreBx, so paths to output files has been added.
* Changed exent of interpolation grid to fit inside bbox of raster arrays
* Moved make_grid to CoreBx_funcs
* Switched names for alongshore and offshore
* Switched to nearest-neighbor interpolation

New in April, 2022
* First four maps are clipped (and then re-clipped to remove sand bars)
* Added lidar


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
# from dask.distributed import LocalCluster
from scipy import interpolate, signal
from CoreBx_funcs import *
%matplotlib inline

In [2]:
drv = 'D:'

In [3]:
# April 9, 2020: Replaced   "2019-09-12-13_1m_DEM_4D_crop.tif",\
# with _v3 and re-ran on my desktop

# May 4, 2020 - Changed to use Jin-Si's merged dems
# Dec 11, 2021 - Switched to unclipped candidate DEMs
# Apr 5, 2022 - Switched to clipped dems for the first four maps
# Apr 26, 2022 - Switched to dems with no sandbars for maps 2 - 4. I had to re-export these from GM so they
#                would encompass the interpolation box, which is why the weird names.

fin_dir = drv+'/crs/proj/2021_FloSupp_Release/Clipped_Final/'
reclip_dir = drv+'/crs/proj/2019_DorianOBX/2022-04-22_nosandbars/'
clip_dir = drv+'/crs/proj/2021_FloSupp_Release/clipfest/dems/'

fnames0 = (\
           fin_dir+'/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif',\
           reclip_dir+'/NCB_2019-09-12-13_clipped_cog_r2.tif',\
           reclip_dir+'/NCB_2019-10-11_clipped_cog_reshape.tif',\
           reclip_dir+'/NCB_2019-11-26_clipped_cog_r3.tif')

fnames1 = (\
clip_dir+'20200208-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
clip_dir+'20200508-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
clip_dir+'20200802_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
clip_dir+'20200805-8_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
clip_dir+'20200928_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif',\
clip_dir+'20210430_Ocracoke_Inlet_to_Cape_LookoutNAD83_2011_NAVD88_UTM18_1m_adj_cog.tif')

fnames=np.append(fnames0,fnames1)

titles = ([\
#         "Oct 6 2018 post-Florence"\
         "Aug 30 2019 pre-Dorian",\
         "Sep 12-13 2019 post-Dorian",\
         "Oct 11 2019 lidar merge",\
         "Nov 26 2019 post-Nor'easter",\
         "Feb 8-9 2020",\
         "May 8-9 2020",\
         "Aug 2 2020 pre-Isaias",\
         "Aug 5- 2020 post-Isaias",\
         "Sep 28 2020 post-Teddy",\
         "May 30 2021"])

dates = ([\
         "2019-08-30",\
         "2019-09-12",\
         "2019-10-11",\
         "2019-11-26",\
         "2020-02-09",\
         "2020-04-09",\
         "2020-08-05",\
         "2020-08-09",\
         "2020-09-28",\
         "2021-05-30"])

nf = len(fnames)
nft = len(titles)
print('Length of datasets',nf,nft)

#TODO - Consider remaking this base file
fill_fnames = ('EBK_201909_YesLidar_Comb_Extent_m.tif')
fill_titles = ('Sep_fill')

# optional median-filter smoothing of original maps
smooth = False
# kernal size...this should be an odd number >= dxy/0.1
ksize = 3

Length of datasets 10 10


In [4]:
# Read in a dict to define the rotated coordinate system
r = yaml2dict('small_island_box.yml')
print(r)

# Make a grid 
xu,yu,xrot,yrot,xcoords,ycoords = make_grid(**r)

ny,nx = np.shape(xu)
print('Size of grid:',ny,nx)

{'name': 'ncorebx_small', 'e0': 383520.0, 'n0': 3860830.0, 'xlen': 25000.0, 'ylen': 1200.0, 'dxdy': 1.0, 'theta': 42.0}
make_grid: Shape of xrot, yrot:  (1200, 25000) (1200, 25000)
corners x, corners y]
[[ 383520.03700711 3860830.70613772]
 [ 402097.91449922 3877558.30216608]
 [ 401295.62690219 3878449.33281183]
 [ 382717.74941009 3861721.73678346]
 [ 383520.03700711 3860830.70613772]]
Saving to ncorebx_small.csv
Size of grid: 1200 25000


In [5]:
%%time
# output for rotated DEMs
dem_path =drv+'/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'

dslist=[]
for i, fn in enumerate(fnames):
    iswarned = False
    fpath = fn
    print(i, fpath)

    # Old way: open the tif with XArray as a DataArray
    #     da = xr.open_rasterio(fpath)
    # New way: use rioxarray
    # The "masked" option puts in NaNs, but takes longer.
    da = rioxarray.open_rasterio( fpath, masked=True )

    print( np.shape(np.flipud(da['y'].values)), np.shape(da['x'].values), np.shape( np.flipud(da.values)) )
    x = da['x'].values
    y = np.flipud(da['y'].values)

    # Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
    # However, make sure to squeeze before flipping
    z = np.flipud(np.squeeze(da.values))
    print(np.shape(x),np.shape(y),np.shape(z))

    if(smooth):
        # smooth with 2D running median
        zs = signal.medfilt2d(z, kernel_size=ksize)
    else:
        zs = z

    f = interpolate.RegularGridInterpolator( (y, x), zs, method='nearest')   

    # Array for interpolated elevations
    zi=np.NaN*np.ones((ny,nx))
        
    # this is the fast iteration, which only works when all of the source points fall inside the target box
    try:
        zi=f((yu,xu))

    # this is a slow iteration through all of the points, but allows us to skip ones that are outside
    except:
        if(not iswarned):
            print("Warning: using slow iteration.")
            iswarned = True
        for ij in np.ndindex(zi.shape):
            try:
                zi[ij]=f((yu[ij],xu[ij]))
            except:
                zi[ij]=np.NaN

    #dar = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
    dar = xr.DataArray(zi,dims=['Cross-shore','Alongshore'],coords={'Cross-shore': ycoords, 'Alongshore' :xcoords })

    dar = dar.chunk()
    dslist.append(dar)

dsa = xr.concat(dslist, dim='map')

0 D:/crs/proj/2021_FloSupp_Release/Clipped_Final//20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83_2011_NAVD88_UTM18N_1m_cog.tif
(24189,) (27433,) (1, 24189, 27433)
(27433,) (24189,) (24189, 27433)
1 D:/crs/proj/2019_DorianOBX/2022-04-22_nosandbars//NCB_2019-09-12-13_clipped_cog_r2.tif
(18803,) (21255,) (1, 18803, 21255)
(21255,) (18803,) (18803, 21255)
2 D:/crs/proj/2019_DorianOBX/2022-04-22_nosandbars//NCB_2019-10-11_clipped_cog_reshape.tif
(19100,) (20940,) (1, 19100, 20940)
(20940,) (19100,) (19100, 20940)
3 D:/crs/proj/2019_DorianOBX/2022-04-22_nosandbars//NCB_2019-11-26_clipped_cog_r3.tif
(19791,) (22379,) (1, 19791, 22379)
(22379,) (19791,) (19791, 22379)
4 D:/crs/proj/2021_FloSupp_Release/clipfest/dems/20200208-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif
(57610,) (48980,) (1, 57610, 48980)
(48980,) (57610,) (57610, 48980)
5 D:/crs/proj/2021_FloSupp_Release/clipfest/dems/20200508-9_Ocracoke_Inlet_to_Cape_Lookout_NAD83_2011_NAVD88_UTM18_1m_adj_cog.tif
(5

In [6]:
dsa

<xarray.DataArray (map: 10, Cross-shore: 1200, Alongshore: 25000)>
dask.array<concatenate, shape=(10, 1200, 25000), dtype=float32, chunksize=(1, 1200, 25000), chunktype=numpy.ndarray>
Coordinates:
  * Cross-shore  (Cross-shore) float64 0.5 1.5 2.5 ... 1.198e+03 1.2e+03
  * Alongshore   (Alongshore) float64 0.5 1.5 2.5 ... 2.5e+04 2.5e+04 2.5e+04
Dimensions without coordinates: map

In [7]:
# TODO - Add some metadata to this netcdf file. Add time to the maps. Are the dimensions in the right order?
fn = dem_path+r['name']+'_clipped.nc'
dsa.to_netcdf(fn)

In [8]:
%%time
# Read in the fill map and make netcdf files
fill_dir = drv+'/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/'
# output for rotated DEMs
dem_path =drv+'/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'

fn = fill_dir+fill_fnames
print(fn)

daf = rioxarray.open_rasterio( fn, masked=True )


print( np.shape(np.flipud(daf['y'].values)), np.shape(daf['x'].values), np.shape( np.flipud(daf.values)) )
x = daf['x'].values
y = np.flipud(daf['y'].values)

# Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
# However, make sure to squeeze before flipping
z = np.flipud(np.squeeze(daf.values))
print(np.shape(x),np.shape(y),np.shape(z))

f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

# Array for interpolated elevations
zi=np.NaN*np.ones((ny,nx))

# this is a slow iteration through all of the points, but allows us to skip ones that are outside
# for ij in np.ndindex(zi.shape):
#     try:
#         zi[ij]=f((yu[ij],xu[ij]))
#     except:
#         zi[ij]=np.NaN

# this is the fast technique.
zi=f((yu,xu))

da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
da = da.chunk()

fno = dem_path+r['name']+'_Sep_fill.nc'
da.to_netcdf(fno)

D:/crs/proj/2019_DorianOBX/Santa_Cruz_Products/merged_dems/EBK_201909_YesLidar_Comb_Extent_m.tif
(26049,) (29111,) (1, 26049, 29111)
(29111,) (26049,) (26049, 29111)
CPU times: total: 18.3 s
Wall time: 21.2 s


In [ ]:
%%time
# Read in the October 90th pctile first-return lidar map and make netcdf files
lidar_dir=drv+'/crs/proj/2019_DorianOBX/DUNEX_lidar/2019 Post Dorian NCMP/'
fn = lidar_dir+'2019_NCMP_PostDorian_CoBa_UTM18_1st90_UTM18_1m_DSM_cog.tif'
print(fn)

daf = rioxarray.open_rasterio( fn, masked=True )

print( np.shape(np.flipud(daf['y'].values)), np.shape(daf['x'].values), np.shape( np.flipud(daf.values)) )
x = daf['x'].values
y = np.flipud(daf['y'].values)

# Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
# However, make sure to squeeze before flipping
z = np.flipud(np.squeeze(daf.values))
print(np.shape(x),np.shape(y),np.shape(z))

f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

# Array for interpolated elevations
zi=np.NaN*np.ones((ny,nx))

# this is the fast technique.
zi=f((yu,xu))

da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
da = da.chunk()

In [8]:
# output for rotated DEMs
dem_path =drv+'/crs/proj/2019_DorianOBX/Dorian_paper_analyses/rotated_dems/'
fno = dem_path+r['name']+'_Oct_lidar_canopy.nc'
da.to_netcdf(fno)

In [9]:
%%time
# Read in the October 90th pctile first-return lidar map and make netcdf files
lidar_dir=drv+'/crs/proj/2019_DorianOBX/DUNEX_lidar/2019 Post Dorian NCMP/'
fn = lidar_dir+'2019_NCMP_PostDorian_CoBa_UTM18_gnd50_UTM18_1m_DSM_cog.tif'
print(fn)

daf = rioxarray.open_rasterio( fn, masked=True )

print( np.shape(np.flipud(daf['y'].values)), np.shape(daf['x'].values), np.shape( np.flipud(daf.values)) )
x = daf['x'].values
y = np.flipud(daf['y'].values)

# Not sure how da.values got a singleton dimension, but squeeze gets rid of it.
# However, make sure to squeeze before flipping
z = np.flipud(np.squeeze(daf.values))
print(np.shape(x),np.shape(y),np.shape(z))

f = interpolate.RegularGridInterpolator( (y, x), z, method='nearest')   

# Array for interpolated elevations
zi=np.NaN*np.ones((ny,nx))

# this is the fast technique.
zi=f((yu,xu))

da = xr.DataArray(zi,dims=['Alongshore','Cross-shore'],coords={'Alongshore': ycoords, 'Cross-shore':xcoords })
da = da.chunk()
fno = dem_path+r['name']+'_Oct_lidar_gnd.nc'
da.to_netcdf(fno)

D:/crs/proj/2019_DorianOBX/DUNEX_lidar/2019 Post Dorian NCMP/2019_NCMP_PostDorian_CoBa_UTM18_gnd50_UTM18_1m_DSM_cog.tif
(58539,) (50750,) (1, 58539, 50750)
(50750,) (58539,) (58539, 50750)
CPU times: total: 2min 15s
Wall time: 2min 16s


In [7]:
da

<xarray.DataArray (Alongshore: 1200, Cross-shore: 25000)>
dask.array<xarray-<this-array>, shape=(1200, 25000), dtype=float32, chunksize=(1200, 25000), chunktype=numpy.ndarray>
Coordinates:
  * Alongshore   (Alongshore) float64 0.5 1.5 2.5 ... 1.198e+03 1.2e+03
  * Cross-shore  (Cross-shore) float64 0.5 1.5 2.5 ... 2.5e+04 2.5e+04 2.5e+04